This notebook will perform a final join between the two cleaned datasets iq_air.csv and pop_clean.csv

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from difflib import get_close_matches

In [74]:
# read air quality data
iq_air = pd.read_csv('iq_air.csv')
iq_air

,cont_rank,city,country,iso3,continent,avg_2019,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,avg_2018,avg_2017
0,1,Hartbeespoort,South Africa,ZAF,Africa,60.0,42.0,65.6,68.5,57.5,112.1,41.1,31.4,59.9,61.0,76.3,21.4,NaN,NaN,NaN
1,2,Bloemfontein,South Africa,ZAF,Africa,42.3,12.6,11.2,NaN,42.8,62.0,102.1,72.8,49.7,33.8,28.9,20.1,16.4,NaN,NaN
2,3,Springs,South Africa,ZAF,Africa,39.1,14.2,13.9,12.2,28.9,66.7,103.4,86.8,50.0,36.3,25.6,11.5,12.3,NaN,NaN
3,4,Vanderbijlpark,South Africa,ZAF,Africa,34.7,NaN,NaN,NaN,25.6,53.9,59.3,56.8,42.0,25.9,20.9,14.8,12.9,NaN,NaN
4,5,Sebokeng,South Africa,ZAF,Africa,32.7,20.8,29.0,31.4,19.1,44.7,56.1,44.6,38.9,29.7,29.0,19.7,23.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4674,80,Tutamandahostel,Ecuador,ECU,South America,6.7,NaN,NaN,NaN,7.1,7.0,6.4,6.7,6.1,7.6,7.6,5.8,6.6,NaN,NaN
4675,81,Calama,Chile,CHL,South America,6.4,6.9,5.0,5.7,7.5,5.1,7.1,6.3,6.7,6.7,6.7,6.3,6.8,13.9,9.8
4676,82,Puerto Baquerizo Moreno,Ecuador,ECU,South America,5.8,NaN,8.7,12.7,7.4,4.3,NaN,3.6,3.3,3.4,2.8,2.3,3.8,NaN,NaN
4677,83,Punta Arenas,Chile,CHL,South America,4.7,3.4,4.1,4.2,4.2,5.5,5.9,5.5,5.1,4.6,4.8,4.6,4.1,4.5,5.5


In [75]:
# read population data
pop = pd.read_csv('pop_clean.csv')
pop

,iso3,country,city,city_alt,lat,lon,pop_2020
0,AFG,Afghanistan,Herat,NaN,34.3482,62.1997,605580
1,AFG,Afghanistan,Kabul,NaN,34.5289,69.1725,4221530
2,AFG,Afghanistan,Kandahar,NaN,31.6133,65.7101,498000
3,AFG,Afghanistan,Mazar-e Sharif,NaN,36.7090,67.1109,532690
4,ALB,Albania,Tiranë,Tirana,41.3275,19.8189,493710
...,...,...,...,...,...,...,...
1855,ZMB,Zambia,Lusaka,NaN,-15.4134,28.2771,2774130
1856,ZMB,Zambia,Ndola,NaN,-12.9587,28.6366,542500
1857,ZWE,Zimbabwe,Bulawayo,NaN,-20.1500,28.5833,638190
1858,ZWE,Zimbabwe,Chitungwiza,NaN,-18.0127,31.0756,386450


After some testing, I decided to implement a near-match (fuzzy match) joining operation instead of a simple inner join. This is because there were hundreds of cities being left out of the join because their spelling was slightly different or the presence of accent markers. This cell will define a fuzzy matching function that will evaluate both the orignal city name and the alternate city name as possible join keys.

In [76]:
# FUZZY MATCHING
def fuzzy_join(city, city_alt, iso3):
    # list of possible cities to match to (within the same country)
    relevant_cities = iq_air[iq_air['iso3']==iso3]
    relevant_cities = list(relevant_cities['city'])
    
    # if perfect match with either city name or alternate name, return matched name
    if city in relevant_cities:
        return city
    elif city_alt in relevant_cities:
        return city_alt
    
    # create list of closest options for both original and alternate city names
    # 0.7 was determined to be a good threshold through multiple rounds of testing
    fuzzy_matches1 = get_close_matches(city,
                                     relevant_cities,
                                     cutoff=0.7)
    fuzzy_matches2 = get_close_matches(str(city_alt),
                                      relevant_cities,
                                      cutoff=0.7)
    
    # return closest fuzzy match if any are found, otherwise return an empty string
    if len(fuzzy_matches1)==0:
        if len(fuzzy_matches2)==0:
            return ''
        else:
            return fuzzy_matches2[0]
    else:
        return fuzzy_matches1[0]

In [77]:
# add column for name determine to match air quality dataset
pop['fuzzy_join_name'] = [
    fuzzy_join(row['city'], row['city_alt'], row['iso3']) \
    for index, row in tqdm(pop.iterrows())
]

In [78]:
# show which rows fuzzy matching was implemented
guesses = pop[(pop['city'] != pop['fuzzy_join_name'])
             & (pop['city_alt'] != pop['fuzzy_join_name'])
             & (pop['fuzzy_join_name'] != '')]
guesses

,iso3,country,city,city_alt,lat,lon,pop_2020,fuzzy_join_name
131,BRA,Brazil,Ribeirão Preto,NaN,-21.1775,-47.8103,714110,Ribeirao Preto
135,BRA,Brazil,São José do Rio Preto,NaN,-20.8197,-49.3794,439020,Sao Jose do Rio Preto
136,BRA,Brazil,São José dos Campos,NaN,-23.1794,-45.8869,721800,Sao Jose dos Campos
137,BRA,Brazil,São Paulo,NaN,-23.5475,-46.6361,22043030,Sao Paulo
139,BRA,Brazil,Taubaté,NaN,-23.0264,-45.5553,312620,Taubate
...,...,...,...,...,...,...,...,...
1790,USA,United States of America,Scranton,NaN,41.4106,-75.6674,375330,Stanton
1799,USA,United States of America,Tampa-St. Petersburg,NaN,27.9475,-82.4584,2877270,St. Petersburg
1801,USA,United States of America,The Woodlands,NaN,30.1630,-95.4558,569770,Woodland
1807,USA,United States of America,Virginia Beach,NaN,36.8345,-76.0872,1477250,Virginia


Although I believe there are a few errors using this method (particularly the Scranton-Stanton match), I believe >95% of the entries returned the proper match.

In [81]:
# join datasets using fuzzy matching column
df1 = pd.merge(iq_air, pop,
               how='inner',
               left_on=['iso3', 'city'],
               right_on=['iso3', 'fuzzy_join_name'])
df1

,cont_rank,city_x,country_x,iso3,continent,avg_2019,jan,feb,mar,apr,...,dec,avg_2018,avg_2017,country_y,city_y,city_alt,lat,lon,pop_2020,fuzzy_join_name
0,2,Bloemfontein,South Africa,ZAF,Africa,42.3,12.6,11.2,NaN,42.8,...,16.4,NaN,NaN,South Africa,Bloemfontein,NaN,-29.1211,26.2140,567030,Bloemfontein
1,6,Kinshasa,Democratic Republic of the Congo,COD,Africa,32.1,NaN,29.9,28.9,27.7,...,19.8,NaN,NaN,Democratic Republic of the Congo,Kinshasa,NaN,-4.3276,15.3136,14342440,Kinshasa
2,9,Accra,Ghana,GHA,Africa,30.3,NaN,NaN,NaN,NaN,...,38.3,NaN,NaN,Ghana,Accra,NaN,5.5560,-0.1969,2514010,Accra
3,10,Vereeniging,South Africa,ZAF,Africa,30.0,14.0,19.7,29.0,21.6,...,23.5,NaN,NaN,South Africa,Vereeniging,NaN,-26.6731,27.9262,774030,Vereeniging
4,12,Kampala,Uganda,UGA,Africa,29.1,35.2,36.9,30.5,17.5,...,30.8,40.8,54.3,Uganda,Kampala,NaN,0.3163,32.5822,3298360,Kampala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,66,Antofagasta,Chile,CHL,South America,12.2,5.3,5.8,8.0,10.5,...,11.0,9.1,10.6,Chile,Antofagasta,NaN,-23.6500,-70.4000,438870,Antofagasta
1004,69,Sao Jose dos Campos,Brazil,BRA,South America,11.8,NaN,NaN,7.8,12.1,...,8.2,NaN,NaN,Brazil,São José dos Campos,NaN,-23.1794,-45.8869,721800,Sao Jose dos Campos
1005,71,Taubate,Brazil,BRA,South America,11.5,10.0,8.9,8.4,12.0,...,8.5,11.4,NaN,Brazil,Taubaté,NaN,-23.0264,-45.5553,312620,Taubate
1006,76,Quito,Ecuador,ECU,South America,8.6,NaN,NaN,NaN,8.7,...,8.7,NaN,NaN,Ecuador,Quito,NaN,-0.2299,-78.5250,1873760,Quito


1,008 rows is not bad, considering the population dataset only had 1,800 rows. With more time I may be able to increase this number

In [83]:
# clean columns
del df1['country_y']
del df1['city_y']
del df1['fuzzy_join_name']
df1 = df1.rename(columns={'city_x':'city',
                         'country_x':'country'})
df1

,cont_rank,city,country,iso3,continent,avg_2019,jan,feb,mar,apr,...,sep,oct,nov,dec,avg_2018,avg_2017,city_alt,lat,lon,pop_2020
0,2,Bloemfontein,South Africa,ZAF,Africa,42.3,12.6,11.2,NaN,42.8,...,33.8,28.9,20.1,16.4,NaN,NaN,NaN,-29.1211,26.2140,567030
1,6,Kinshasa,Democratic Republic of the Congo,COD,Africa,32.1,NaN,29.9,28.9,27.7,...,26.8,18.7,17.5,19.8,NaN,NaN,NaN,-4.3276,15.3136,14342440
2,9,Accra,Ghana,GHA,Africa,30.3,NaN,NaN,NaN,NaN,...,22.1,20.6,30.9,38.3,NaN,NaN,NaN,5.5560,-0.1969,2514010
3,10,Vereeniging,South Africa,ZAF,Africa,30.0,14.0,19.7,29.0,21.6,...,29.2,27.1,19.9,23.5,NaN,NaN,NaN,-26.6731,27.9262,774030
4,12,Kampala,Uganda,UGA,Africa,29.1,35.2,36.9,30.5,17.5,...,32.8,18.1,24.1,30.8,40.8,54.3,NaN,0.3163,32.5822,3298360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,66,Antofagasta,Chile,CHL,South America,12.2,5.3,5.8,8.0,10.5,...,11.1,11.8,10.4,11.0,9.1,10.6,NaN,-23.6500,-70.4000,438870
1004,69,Sao Jose dos Campos,Brazil,BRA,South America,11.8,NaN,NaN,7.8,12.1,...,14.2,10.4,8.0,8.2,NaN,NaN,NaN,-23.1794,-45.8869,721800
1005,71,Taubate,Brazil,BRA,South America,11.5,10.0,8.9,8.4,12.0,...,14.3,11.0,8.4,8.5,11.4,NaN,NaN,-23.0264,-45.5553,312620
1006,76,Quito,Ecuador,ECU,South America,8.6,NaN,NaN,NaN,8.7,...,10.8,10.2,8.5,8.7,NaN,NaN,NaN,-0.2299,-78.5250,1873760


Looks ready to be dashboarded!

In [85]:
# write to csv
df1.to_csv('final_data.csv', index=False)